# 9. 데이터 가공
## 9.1 집단별로 함수 적용하기

In [1]:
import rpy2.rinterface

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
v <- c(172, 172, 170, 170, rnorm(96, 170, 3))
g <- rep(c("Male", "Female"), 50)
c(mean(v[g=="Male"]), mean(v[g=="Female"]))

[1] 170.6673 169.9694


In [4]:
import numpy as np
import pandas as pd

v = pd.Series([172, 172, 170, 170] + np.random.normal(170, 3, 96).tolist())
g = pd.Series(['Male', 'Female']*50)
v[g=="Male"].mean(), v[g=="Female"].mean()

(170.19860073821798, 170.0825118766395)

In [5]:
%%R -i v
g <- rep(c("Male", "Female"), 50)
c(mean(v[g=="Male"]), mean(v[g=="Female"]))

[1] 170.1986 170.0825


In [6]:
%%R
set.seed(0)
v <- rnorm(1000, 170, 3)
g <- rep(1:100, 10)
c(mean(v[g==1]), mean(v[g==2]), mean(v[g==3]), mean(v[g==4]))

[1] 169.0207 169.1349 171.9850 171.0578


In [7]:
np.random.seed(0)
v = np.random.normal(170, 3, 1000)
g = np.array(list(range(1,100+1,1))*10)
(v[g==1].mean(), v[g==2].mean(), v[g==3].mean(), v[g==4].mean())

(169.61289280337203, 169.5890450809177, 170.3389388587738, 171.33178654345318)

In [8]:
%%R
v = c(2, 2, 6, 8, 9, 1, 2, 7, 5) 
g = c(1, 3, 1, 2, 3, 2, 2, 1, 2) 
tapply(v, g, FUN = mean) 

  1   2   3 
5.0 4.0 5.5 


In [9]:
v = pd.Series([2, 2, 6, 8, 9, 1, 2, 7, 5])
g = pd.Series([1, 3, 1, 2, 3, 2, 2, 1, 2])
pd.DataFrame({'v':v, 'g':g}).groupby('g').mean()

,v
g,
1,5.0
2,4.0
3,5.5


In [10]:
%%R -o dat
dat <- data.frame(gender=c('M','M','M','M','M','F','F','F','F','F'),
                  num=c(1,2,3,1,2,3,1,2,3,1),
                  h=c(170,180,190,180,170,150,160,170,160,150),
                  w=c(80,70,100,80,60,50,50,60,60,50))
dat$BMI <- dat$w/(dat$h/100)^2
table(dat$gender, dat$num)

   
    1 2 3
  F 2 1 2
  M 2 2 1


In [12]:
%%R 
tapply(dat$h, list(dat$gender, dat$num), mean)


#https://stackoverflow.com/questions/53781634/aggregation-in-pandas
#https://stackoverflow.com/questions/17621325/equivalent-pandas-function-to-this-r-aggregation

    1   2   3
F 155 170 155
M 175 175 190


In [13]:
%%R 
aggregate(h~gender+num, sum, data=dat)

  gender num   h
1      F   1 310
2      M   1 350
3      F   2 170
4      M   2 350
5      F   3 310
6      M   3 190


In [14]:
%%R 
aggregate(h+w~gender+num, sum, data=dat)

  gender num h + w
1      F   1   410
2      M   1   510
3      F   2   230
4      M   2   480
5      F   3   420
6      M   3   290


In [15]:
%%R 
aggregate(cbind(h,w)~gender+num, sum, data=dat)

  gender num   h   w
1      F   1 310 100
2      M   1 350 160
3      F   2 170  60
4      M   2 350 130
5      F   3 310 110
6      M   3 190 100


In [16]:
%%R 
aggregate(.~gender+num, sum, data=dat)

  gender num   h   w      BMI
1      F   1 310 100 41.75347
2      M   1 350 160 52.37302
3      F   2 170  60 20.76125
4      M   2 350 130 42.36618
5      F   3 310 110 45.65972
6      M   3 190 100 27.70083


In [17]:
%%R 
aggregate(dat, list(dat$gender, dat$num), length)

  Group.1 Group.2 gender num h w BMI
1       F       1      2   2 2 2   2
2       M       1      2   2 2 2   2
3       F       2      1   1 1 1   1
4       M       2      2   2 2 2   2
5       F       3      2   2 2 2   2
6       M       3      1   1 1 1   1


In [18]:
dat.head()

,gender,num,h,w,BMI
1,M,1.0,170.0,80.0,27.681661
2,M,2.0,180.0,70.0,21.604938
3,M,3.0,190.0,100.0,27.700831
4,M,1.0,180.0,80.0,24.691358
5,M,2.0,170.0,60.0,20.761246


In [19]:
dat['BMI'] = dat['w']/(dat['h']/100)**2

In [20]:
dat['gender'].value_counts() # value_counts

M    5
F    5
Name: gender, dtype: int64

In [21]:
dat.loc[:, ['gender', 'num', 'w']].groupby(['gender', 'num']).count().unstack('num')

w        
num    1.0 2.0 3.0
gender            
F        2   1   2
M        2   2   1

In [22]:
#tapply(dat$h, list(dat$gender, dat$num), mean)
dat[['h', 'gender', 'num']].groupby(['num', 'gender']).mean().unstack('num')

h              
num       1.0    2.0    3.0
gender                     
F       155.0  170.0  155.0
M       175.0  175.0  190.0

In [23]:
#aggregate(h~gender+num, sum, data=dat)
dat[['h', 'gender', 'num']].groupby(['gender', 'num']).sum()

h
gender num       
F      1.0  310.0
       2.0  170.0
       3.0  310.0
M      1.0  350.0
       2.0  350.0
       3.0  190.0

In [24]:
#aggregate(h+w~gender+num, sum, data=dat)
dat['h_plus_w'] = dat['h']+dat['w']
dat[['h_plus_w', 'gender', 'num']].groupby(['gender', 'num']).sum()

h_plus_w
gender num          
F      1.0     410.0
       2.0     230.0
       3.0     420.0
M      1.0     510.0
       2.0     480.0
       3.0     290.0

In [25]:
#aggregate(cbind(h,w)~gender+num, sum, data=dat)
dat[['h', 'w', 'gender', 'num']].groupby(['gender', 'num']).sum()

h      w
gender num              
F      1.0  310.0  100.0
       2.0  170.0   60.0
       3.0  310.0  110.0
M      1.0  350.0  160.0
       2.0  350.0  130.0
       3.0  190.0  100.0

In [26]:
#aggregate(.~gender+num, sum, data=dat)
dat.groupby(['gender', 'num']).sum()

h      w        BMI  h_plus_w
gender num                                   
F      1.0  310.0  100.0  41.753472     410.0
       2.0  170.0   60.0  20.761246     230.0
       3.0  310.0  110.0  45.659722     420.0
M      1.0  350.0  160.0  52.373019     510.0
       2.0  350.0  130.0  42.366184     480.0
       3.0  190.0  100.0  27.700831     290.0

In [27]:
#aggregate(dat, list(dat$gender, dat$num), length)
dat.groupby(['gender', 'num']).apply(lambda x: len(x))

gender  num
F       1.0    2
        2.0    1
        3.0    2
M       1.0    2
        2.0    2
        3.0    1
dtype: int64

In [28]:
dat.describe()

,num,h,w,BMI,h_plus_w
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,1.900000,168.000000,66.000000,23.061447,234.000000
std,0.875595,13.165612,16.465452,2.833185,28.751812
min,1.000000,150.000000,50.000000,19.531250,200.000000
25%,1.000000,160.000000,52.500000,20.972169,212.500000
50%,2.000000,170.000000,60.000000,22.222222,230.000000
75%,2.750000,177.500000,77.500000,24.377894,250.000000
max,3.000000,190.000000,100.000000,27.700831,290.000000


In [30]:
%%R
by(dat, list(dat$gender, dat$num), summary)[1:2]

[[1]]
 gender      num          h               w           BMI       
 F:2    Min.   :1   Min.   :150.0   Min.   :50   Min.   :19.53  
 M:0    1st Qu.:1   1st Qu.:152.5   1st Qu.:50   1st Qu.:20.20  
        Median :1   Median :155.0   Median :50   Median :20.88  
        Mean   :1   Mean   :155.0   Mean   :50   Mean   :20.88  
        3rd Qu.:1   3rd Qu.:157.5   3rd Qu.:50   3rd Qu.:21.55  
        Max.   :1   Max.   :160.0   Max.   :50   Max.   :22.22  

[[2]]
 gender      num          h               w           BMI       
 F:0    Min.   :1   Min.   :170.0   Min.   :80   Min.   :24.69  
 M:2    1st Qu.:1   1st Qu.:172.5   1st Qu.:80   1st Qu.:25.44  
        Median :1   Median :175.0   Median :80   Median :26.19  
        Mean   :1   Mean   :175.0   Mean   :80   Mean   :26.19  
        3rd Qu.:1   3rd Qu.:177.5   3rd Qu.:80   3rd Qu.:26.93  
        Max.   :1   Max.   :180.0   Max.   :80   Max.   :27.68  



In [31]:
datDescribe = dat.groupby(['gender', 'num']).apply(lambda x: x.describe())
datDescribe.head(n =10)

num           h     w        BMI    h_plus_w
gender num                                                    
F      1.0 count  2.0    2.000000   2.0   2.000000    2.000000
           mean   1.0  155.000000  50.0  20.876736  205.000000
           std    0.0    7.071068   0.0   1.902805    7.071068
           min    1.0  150.000000  50.0  19.531250  200.000000
           25%    1.0  152.500000  50.0  20.203993  202.500000
           50%    1.0  155.000000  50.0  20.876736  205.000000
           75%    1.0  157.500000  50.0  21.549479  207.500000
           max    1.0  160.000000  50.0  22.222222  210.000000
       2.0 count  1.0    1.000000   1.0   1.000000    1.000000
           mean   2.0  170.000000  60.0  20.761246  230.000000